In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc
#import cupy
#import cudf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet


In [2]:
def load_data(path=''):
    df = pd.read_parquet(path)
    #df['customer_ID'] = df['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    df.S_2 = pd.to_datetime(df.S_2, errors='coerce')
    df = df.sort_values(['customer_ID','S_2'])
    
    df['S_2_dateofweek'] = df['S_2'].dt.weekday
    df['S_2_dateofmonth'] = df['S_2'].dt.day
    
    null_vals = df.isnull().sum() / df.shape[0]
    print(null_vals.head())
    null_cols = null_vals[null_vals>0.3].index.values
    for col in null_cols:
        df[col + 'null'] = df[col].isnull().astype(int)
    df = df.drop(columns=['S_2'])
    return df

In [3]:
def feature_engineer(df):
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
    cat1_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    cat2_features = [
                    'B_31','B_32','B_33','D_103','D_109','D_111','D_127',
                    'D_129','D_135','D_137','D_139','D_140','D_143','D_86',
                    'D_87','D_92','D_93','D_94','D_96','R_15','R_19','R_2','R_21',
                    'R_22','R_23','R_24','R_25','R_28','R_4','S_18','S_20','S_6'
                       ]
    cat3_features = [
                    'R_9','R_18','R_10','R_11','D_89','D_91','D_81','D_82','D_136',
                    'D_138','D_51','D_123','D_125','D_108','B_41','B_22',
                       ]
    null_values = df.isnull().sum()/df.shape[0]
    print('type null_values',type(null_values))
    exnullcols = null_values[null_values>0.9].index.tolist()
    print('type exnullcols',type(exnullcols))
    print(exnullcols)
    nullcols = null_values[null_values>0.3].index.tolist()
    print('type nullcols',type(nullcols))
    print(nullcols)
    nullprefixcols = [col+'null' for col in nullcols]
    
    cat_cols = cat1_features + cat2_features + cat3_features + exnullcols + nullprefixcols
    num_cols = [col for col in all_cols if col not in cat_cols]
    
    num_df = df.groupby('customer_ID')[num_cols].agg(['first','mean','std','min','max','last'])
    num_df.columns = ['_'.join(col) for col in num_df.columns]
    
    cat1_df = df.groupby('customer_ID')[cat1_features].agg(['first','last','nunique'])
    cat1_df.columns = ['_'.join(col) for col in cat1_df.columns]
    cat2_df = df.groupby('customer_ID')[cat2_features].agg(['first','last','nunique'])
    cat2_df.columns = ['_'.join(col) for col in cat2_df.columns]
    cat3_df = df.groupby('customer_ID')[cat3_features].agg(['first','last','nunique','mean','std','min','max'])
    cat3_df.columns = ['_'.join(col) for col in cat3_df.columns]
    cat4_df = df.groupby('customer_ID')[exnullcols].agg(['last'])
    cat4_df.columns = ['_'.join(col) for col in cat4_df.columns]
    cat5_df = df.groupby('customer_ID')[nullprefixcols].agg(['count'])
    cat5_df.columns = ['_'.join(col) for col in cat5_df.columns]
    
    temp1 = df.groupby('customer_ID')['P_2'].agg(['count'])
    #temp1.reset_index()
    temp1.columns = ['nstatements']
    #temp1 = temp1.set_index('customer_ID')
    print(temp1.head())
    
    df = pd.concat([num_df,cat1_df,cat2_df,cat3_df,cat4_df,cat5_df,temp1],axis=1)
    del num_df,cat1_df,cat2_df,cat3_df,cat4_df,cat5_df,temp1
    _ = gc.collect()
    return df
    

In [4]:
def denoise(df):
    for col in tqdm(df.columns):
        if col not in ['customer_ID','S_2']:
            df[col] = np.floor(df[col]*100)
    return df
def one_hot_encoding(df,cols,del_col=True):
    print(df.shape)
    for col in cols:
        dummies = pd.get_dummies(df[col],prefix='onehot_%s'%col)
        df = pd.concat([df,dummies],axis=1)
    print(df.shape)
    if del_col == True:
        df.drop(cols,axis=1,inplace=True)
    print(df.shape)
    return df
def categ_feature(df):
    col_one_hot = [col for col in df.columns if 'onehot' in col]
    if lastk is None:
        one_hot_df = df.groupby('customer_ID',sort=False)[col_one_hot].agg(['mean','std','sum','last'])
    else:
        one_hot_df = df.groupby('customer_ID',sort=False)[col_one_hot].agg(['mean','std','sum'])
    if lastk is None:
        cat_df = df.groupby('customer_ID',sort=False)[cat_feature].agg(['last','nunique'])
    else:
        cat_df = df.groupby('customer_ID',sort=False)[cat_feature].agg(['nunique'])
    day_df = df.groupby('customer_ID',sort=False)['S_2'].agg('count')
    df = df.concat([one_hot_df,cat_df,day_df],axis=1)
    return df
def num_feature(df):
    if num_features[0][:5]=='rank_':
        num_df = df.groupby('customer_ID',sort=False)[num_features].agg(['last'])
    else:
        if lastk is None:
            num_df = df.groupby('customer_ID',sort=False)[num_features].agg(['mean','std','max','min','sum','last'])
        else:
            num_df = df.groupby('customer_ID',sort=False)[num_features].agg(['mean','std','max','min','sum'])
        for col in num_df.columns:
            num_df[col] = num_df[col]//0.01
    df = num_df.reset_index()
    return df
def diff_feature(df):
    diff_features = [f'diff_{col}' for col in num_features]
    cus = df['customer_ID'].values
    df = df.groupby('customer_ID',sort=False)[num_features].diff().add_prefix('diff_')
    df.insert(0,'customer_ID',cus)
    if lastk in None:
        diff_df = df.groupby('customer_ID',sort=False)[diff_features].agg(['mean','std','min','max','sum','last'])
    else:
        diff_df = df.groupby('customer_ID',sort=False)[diff_features].agg(['mean','std','min','max','sum'])
    for col in diff_df.columns:
        diff_df[col] = diff_df[col]//0.01
    diff_df.reset_index()
    return df

In [5]:
train = load_data('/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet')

customer_ID    0.000000
S_2            0.000000
P_2            0.008313
D_39           0.000000
B_1            0.000000
dtype: float64


In [6]:
train = feature_engineer(train)

type null_values <class 'pandas.core.series.Series'>
type exnullcols <class 'list'>
['D_73', 'B_29', 'D_88', 'D_110', 'B_39', 'B_42', 'D_132', 'D_134']
type nullcols <class 'list'>
['D_42', 'D_50', 'D_53', 'D_56', 'S_9', 'B_17', 'D_73', 'D_76', 'D_77', 'B_29', 'D_88', 'D_105', 'D_110', 'B_39', 'B_42', 'D_132', 'D_134', 'D_142']
                                                    nstatements
customer_ID                                                    
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fb...           13
00000fd6641609c6ece5454664794f0340ad84dddce9a26...           13
00001b22f846c82c51f6e3958ccd81970162bae8b007e80...           13
000041bdba6ecadd89a52d11886e8eaaec9325906c97233...           13
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad...           13


In [7]:
train_label = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv')
submission = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')

In [8]:
test = load_data('/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet')

customer_ID    0.000000
S_2            0.000000
P_2            0.005269
D_39           0.000000
B_1            0.000000
dtype: float64


In [9]:
test = feature_engineer(test)

type null_values <class 'pandas.core.series.Series'>
type exnullcols <class 'list'>
['D_73', 'D_88', 'D_110', 'B_39', 'B_42', 'D_134']
type nullcols <class 'list'>
['D_42', 'D_43', 'D_50', 'D_53', 'D_56', 'S_9', 'B_17', 'D_73', 'D_76', 'D_77', 'B_29', 'D_88', 'D_105', 'D_110', 'B_39', 'B_42', 'D_132', 'D_134', 'D_142']
                                                    nstatements
customer_ID                                                    
00000469ba478561f23a92a868bd366de6f6527a684c9a2...            9
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397...           13
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5...           13
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf...           13
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a...           13


In [10]:
train_label.reset_index()
train_label = train_label.set_index('customer_ID')
train = train.merge(train_label,left_index=True,right_index=True,how='left')

In [11]:
print(train.shape)
print(test.shape)

(458913, 1007)
(924621, 1017)


In [12]:
all_cols = [col for col in train.columns if col in test.columns]

In [13]:
train.head()

,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,...,D_88null_count,D_105null_count,D_110null_count,B_39null_count,B_42null_count,D_132null_count,D_134null_count,D_142null_count,nstatements,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.938469,0.933824,0.024194,0.868580,0.960384,0.934745,0,0.230769,0.832050,0,...,13,13,13,13,13,13,13,13,13,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.929122,0.899820,0.022119,0.861109,0.929122,0.880519,13,7.153846,6.743468,0,...,13,13,13,13,13,13,13,13,13,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.876615,0.878454,0.028911,0.797670,0.904482,0.880875,0,0.000000,0.000000,0,...,13,13,13,13,13,13,13,13,13,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.567442,0.598969,0.020107,0.567442,0.623392,0.621776,9,1.538462,3.017046,0,...,13,13,13,13,13,13,13,13,13,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.936842,0.891679,0.042325,0.805045,0.940382,0.871900,0,0.000000,0.000000,0,...,13,13,13,13,13,13,13,13,13,0


In [14]:
len(all_cols)

1006

#Preparing the data

In [15]:
#from sklearn.preprocessing import LabelEncoder
#train['month'] = pd.DatetimeIndex(train['S_2']).month
#test['month'] = pd.DatetimeIndex(test['S_2']).month

In [16]:
#train = pd.merge(train,train_label,how='left',on='customer_ID')

In [17]:
#train['target'].value_counts()

In [18]:
#columns_feature = train.columns.drop(['customer_ID','S_2','target']).values

In [19]:
#from sklearn.preprocessing import MinMaxScaler
#scal = MinMaxScaler().fit(train[columns_feature])

In [20]:
import lightgbm as lgm
model_select = lgm.LGBMClassifier()
model_select.fit(train[all_cols],train['target'])
import_cols = model_select.booster_.feature_importance()

In [21]:
feat_imp = pd.DataFrame({'col':train[all_cols].columns,'imp':import_cols})

In [22]:
feat_imp = feat_imp.sort_values(by=['imp'],ascending = False)

In [23]:
feat_imp

,col,imp
5,P_2_last,164
11,D_39_last,92
47,B_3_last,75
71,B_4_last,70
89,D_46_last,52
...,...,...
494,D_83_std,0
492,D_83_first,0
491,R_13_last,0
490,R_13_max,0


In [24]:
#all_cols = feat_imp['col'].values[:500]

In [25]:
from sklearn.model_selection import train_test_split
train_data, valid_data, train_label, valid_label = train_test_split(train[all_cols],train['target'],test_size=0.2,random_state=27,stratify=train['target'])

# Train models

## Model Lightgbm

In [26]:
import lightgbm as lgm
model1 = lgm.LGBMClassifier()
model1.fit(train_data, train_label)

LGBMClassifier()

In [27]:
pred1 = model1.predict(valid_data)

In [28]:
from sklearn.metrics import f1_score
sco1 = f1_score(valid_label,pred1)

In [29]:
print(sco1)

0.8078000250804664


In [30]:
test['prediction1'] = model1.predict(test.loc[:,all_cols])

/tmp/ipykernel_21/4214335181.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['prediction1'] = model1.predict(test.loc[:,all_cols])


In [31]:
test.head()

,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,...,D_88null_count,D_105null_count,D_110null_count,B_39null_count,B_42null_count,D_132null_count,D_134null_count,D_142null_count,nstatements,prediction1
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.631315,0.601387,0.020190,0.568930,0.631315,0.568930,0,2.222222,3.527668,0,...,9,9,9,9,9,9,9,9,9,0
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.894195,0.862166,0.031436,0.794469,0.913501,0.841177,11,5.076923,6.034091,0,...,13,13,13,13,13,13,13,13,13,0
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.774276,0.748955,0.061456,0.673112,0.835114,0.697522,19,6.000000,9.000000,0,...,13,13,13,13,13,13,13,13,13,0
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.514222,0.474728,0.028856,0.428457,0.514222,0.513186,14,15.846154,4.355957,7,...,13,13,13,13,13,13,13,13,13,0
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.286302,0.324100,0.049865,0.254478,0.425764,0.254478,6,11.846154,6.681394,1,...,13,13,13,13,13,13,13,13,13,1


## Model XGBoost

In [32]:
from xgboost import XGBClassifier

In [33]:
model2 = XGBClassifier().fit(train_data, train_label)

In [34]:
pred2 = model2.predict(valid_data)

In [35]:
from sklearn.metrics import f1_score
sco2 = f1_score(valid_label,pred2)

In [36]:
print(sco2)

0.8049057556378324


In [37]:
test['prediction2'] = model2.predict(test.loc[:,all_cols])

/tmp/ipykernel_21/3369942069.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['prediction2'] = model2.predict(test.loc[:,all_cols])


## Model DNN

In [38]:
import tensorflow as tf
from tensorflow import keras

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [39]:
train_data.shape

(367130, 1006)

In [40]:
model3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(1006,)),
    keras.layers.Dense(16,activation = 'relu'),
    keras.layers.Dense(16,activation = 'relu'),
    keras.layers.Dense(1,activation = 'sigmoid')])

In [41]:
model3.compile(optimizer='adam',loss='binary_crossentropy', metrics = 'accuracy')

In [42]:
train_data = train_data.fillna(0)
valid_data = valid_data.fillna(0)
train_label = train_label.fillna(0)
valid_label = valid_label.fillna(0)
test = test.fillna(0)

In [43]:
model3.fit(train_data, train_label,epochs=1,batch_size = 32)

11473/11473 [==============================] - 19s 2ms/step - loss: 0.2981 - accuracy: 0.8802


In [44]:
pred3 = model3.predict(valid_data,batch_size = 64, verbose = 1).round()

1435/1435 [==============================] - 2s 1ms/step


In [45]:
#pred3 = np.argmax(pred3,axis=1)

In [46]:
print(pred3)

[[1.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


In [47]:
from sklearn.metrics import classification_report
print(classification_report(valid_label,pred3))

              precision    recall  f1-score   support

           0       0.95      0.88      0.92     68017
           1       0.73      0.88      0.80     23766

    accuracy                           0.88     91783
   macro avg       0.84      0.88      0.86     91783
weighted avg       0.90      0.88      0.89     91783



In [48]:
test = test.reset_index()

In [49]:
test.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,...,D_105null_count,D_110null_count,B_39null_count,B_42null_count,D_132null_count,D_134null_count,D_142null_count,nstatements,prediction1,prediction2
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.631315,0.601387,0.020190,0.568930,0.631315,0.568930,0,2.222222,3.527668,...,9,9,9,9,9,9,9,9,0,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.894195,0.862166,0.031436,0.794469,0.913501,0.841177,11,5.076923,6.034091,...,13,13,13,13,13,13,13,13,0,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.774276,0.748955,0.061456,0.673112,0.835114,0.697522,19,6.000000,9.000000,...,13,13,13,13,13,13,13,13,0,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.514222,0.474728,0.028856,0.428457,0.514222,0.513186,14,15.846154,4.355957,...,13,13,13,13,13,13,13,13,0,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.286302,0.324100,0.049865,0.254478,0.425764,0.254478,6,11.846154,6.681394,...,13,13,13,13,13,13,13,13,1,1


In [50]:
for i in range(9):
    test.loc[100000*i:100000*(i+1),['prediction3']] = model3.predict(test.loc[100000*i:100000*(i+1),all_cols]).round()

3126/3126 [==============================] - 3s 1ms/step


In [51]:
test.loc[900000:,['prediction3']] = model3.predict(test.loc[900000:,all_cols]).round()

770/770 [==============================] - 1s 1ms/step


In [52]:
#predict = test[['customer_ID','prediction']].groupby('customer_ID')['prediction'].mean().round()

In [53]:
#test['prediction'] = model3.predict(test.loc[:,all_cols])

# Ensembled

In [54]:
test['prediction'] = test['prediction1']*0.3 + test['prediction2']*0.4 + test['prediction3']*0.2
#test['prediction'] = test['prediction1']*0.5 + test['prediction3']*0.5

/tmp/ipykernel_21/2268524996.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['prediction'] = test['prediction1']*0.3 + test['prediction2']*0.4 + test['prediction3']*0.2


In [55]:
test['prediction'] = test['prediction'].round()

In [56]:
predict = test[['customer_ID','prediction']]

In [57]:
predict['prediction'].isnull().sum()

0

In [58]:
predict.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,1.0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1.0


In [59]:
predict = predict.astype({'prediction':int})

In [60]:
predict.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1


In [61]:
submission1 = pd.merge(submission['customer_ID'],predict,how='left',on='customer_ID')

In [62]:
submission1.to_csv('submission.csv',index=False)

In [63]:
#column_float = train.select_dtypes(include=['float32']).columns
#train[column_float] = train[column_float].astype(np.float16)
#test[column_float] = test[column_float].astype(np.float16)

In [64]:
submission1.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1
